In [3]:
import keras
import sys

import numpy as np
import pandas as pd 
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [6]:
train_data_dir = "/home/linnik/My_dataset_new/train"   
validation_data_dir = "/home/linnik/My_dataset_new/val"

In [7]:
img_width, img_height = 71, 71
nb_train_samples = 8403
nb_validation_samples = 502
epochs = 50
batch_size = 8
n_classes = 11

In [8]:
K.set_learning_phase(1)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    zoom_range=0.2,
    rotation_range = 5,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

Found 8403 images belonging to 11 classes.
Found 502 images belonging to 11 classes.


In [11]:
def build_model_Xception():
    base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = n_classes)
  
    for layer in base_model.layers:
        layer.trainable = True
      
    model = Model(inputs = base_model.input, outputs = base_model.output)
    return model

In [12]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [13]:
early_stop = EarlyStopping(monitor = 'val_loss', patience = 8, verbose = 1, min_delta = 1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-4)
check = ModelCheckpoint("Best_Xception_50_epochs.hdf5", monitor = "val_acc", save_best_only = True) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr, check]

In [14]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
760/760 [==============================] - 51s 67ms/step - loss: 1.1297 - acc: 0.6157 - val_loss: 3.5010 - val_acc: 0.4677
Epoch 2/50
760/760 [==============================] - 35s 46ms/step - loss: 0.4327 - acc: 0.8396 - val_loss: 3.1885 - val_acc: 0.5405
Epoch 3/50
760/760 [==============================] - 39s 52ms/step - loss: 0.2003 - acc: 0.9354 - val_loss: 3.7807 - val_acc: 0.5405
Epoch 4/50
760/760 [==============================] - 37s 49ms/step - loss: 0.1653 - acc: 0.9474 - val_loss: 3.6922 - val_acc: 0.5769
Epoch 5/50
760/760 [==============================] - 38s 50ms/step - loss: 0.1013 - acc: 0.9651 - val_loss: 4.0177 - val_acc: 0.5425
Epoch 6/50
760/760 [==============================] - 39s 52ms/step - loss: 0.0992 - acc: 0.9717 - val_loss: 5.1287 - val_acc: 0.4130

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 7/50
760/760 [

In [15]:
loaded_model = load_model("Best_Xception_50_epochs.hdf5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
right = 0
wrong = 0
times = []

for folder in os.listdir(validation_data_dir):
    for img in os.listdir(validation_data_dir + "/" + folder):
        imag = im.load_img(validation_data_dir + "/" + folder + "/" + img, target_size = (img_width, img_height))
        
        photo_ar = im.img_to_array(imag)
        photo_ar = np.expand_dims(photo_ar, axis = 0)
        photo_ar /= 255
        
        start_time = time.time()
        ans = np.argmax(loaded_model.predict(photo_ar))
        times.append(time.time() - start_time)
        
        if ans == int(folder)-1:
            right += 1
        else:
            wrong += 1

In [23]:
print("Accuracy on validation set is", right/(right + wrong))
print("Average time to pridict one photo is", np.array(times).mean(), "seconds")

Accuracy on validation set is 0.7888446215139442
Average time to pridict one photo is 0.01221259656655361 seconds


In [31]:
# Проверка качества сети по классам (Precision and Recall)

TP = [0 for i in range(n_classes)]
FN = [0 for i in range(n_classes)]
FP = [0 for i in range(n_classes)]

precision = [0 for i in range(n_classes)]
recall = [0 for i in range(n_classes)]

for folder in os.listdir(validation_data_dir):
    for img in os.listdir(validation_data_dir + "/" + folder):
        imag = im.load_img(validation_data_dir + "/" + folder + "/" + img, target_size = (img_width, img_height))
        
        photo_ar = im.img_to_array(imag)
        photo_ar = np.expand_dims(photo_ar, axis = 0)
        photo_ar /= 255
        
        ans = np.argmax(loaded_model.predict(photo_ar))
    
        if ans == int(folder)-1:
            TP[ans] += 1
        if ans != int(folder)-1:
            FN[ans] += 1
            FP[int(folder)-1] += 1

In [32]:
for i in range(n_classes):
    try:
        precision[i] = TP[i]/(TP[i] + FP[i])
    except:
        precision[i] = 0
        
    try:
        recall[i] = TP[i]/(TP[i] + FN[i])
    except:
        recall[i] = 0
    
precision = np.array(precision)
recall = np.array(recall)

print(precision.max(), precision.min(), precision.mean())
print(recall.max(), recall.min(), recall.mean())

1.0 0.0 0.6574384396965042
1.0 0.0 0.5896097521136655
